In [1]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np
from pandas_datareader import data, wb

In [2]:
from datetime import date, timedelta, datetime
from time import sleep
# 08112015
import pytz

In [3]:
s_p_500=pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies', attrs={"class":'wikitable sortable'}, header=0)[0]

In [4]:
s_p_500

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877
5,AYI,Acuity Brands Inc,reports,Industrials,Electrical Components & Equipment,"Atlanta, Georgia",2016-05-03,1144215
6,ADBE,Adobe Systems Inc,reports,Information Technology,Application Software,"San Jose, California",NaN,796343
7,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Roanoke, Virginia",2015-07-09,1158449
8,AES,AES Corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",NaN,874761
9,AET,Aetna Inc,reports,Health Care,Managed Health Care,"Hartford, Connecticut",1976-06-30,1122304


In [5]:
s_p_500.columns = [x.strip().replace(' ', '_') for x in s_p_500.columns]

In [6]:
industries=s_p_500.GICS_Sector.unique()

In [7]:
companies=[]
for industry in industries:
    companies.append(s_p_500[(s_p_500.GICS_Sector==industry)].head(3))

In [8]:
companies = pd.concat(companies, axis=0)

In [9]:
companies.head()

,Ticker_symbol,Security,SEC_filings,GICS_Sector,GICS_Sub_Industry,Address_of_Headquarters,Date_first_added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
5,AYI,Acuity Brands Inc,reports,Industrials,Electrical Components & Equipment,"Atlanta, Georgia",2016-05-03,1144215
15,ALK,Alaska Air Group Inc,reports,Industrials,Airlines,"Seattle, Washington",2016-05-13,766421
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152


In [10]:
companies.Ticker_symbol = companies.Ticker_symbol.apply(lambda x: x.replace('-', ''))

In [77]:
def get_links_for_symbol(symbol, start_date=date(2014, 1, 1), end_date = date(2014, 1, 15)):
    def get_link(href, text, date):
        link={}
        link['url']="http://www.reuters.com/"+href
        link['text']=text
        link['symbol']=symbol
        link['date']=date
        link['article']=BeautifulSoup(urllib2.urlopen(link['url']), "lxml").find_all(
            'span', {"id":"article-text"})[0].text.replace('\n', ' ')
#         sleep(0.5)
        return link

    links=[]
    for day in [start_date + timedelta(n) for n in range((end_date - start_date).days)]:
        url="http://www.reuters.com/finance/stocks/companyNews?symbol={}&date={}".format(symbol,day.strftime('%m%d%Y'))
        news = BeautifulSoup(urllib2.urlopen(url), "lxml").find_all('div', {"id":"companyNews"})
        for i in xrange(2):
            for feature in news[i].find_all('h2'):
                a=feature.find('a')
                if a.has_attr('href'):
                    links.append(get_link(a['href'], a.text, day))
        sleep(1)
    return links

In [32]:
# links=[]
# for symbol in companies.Ticker_symbol:
#     links += get_links_for_symbol(symbol)

In [78]:
links = get_links_for_symbol('T')

In [79]:
links=pd.DataFrame(links)

In [80]:
links

,article,date,symbol,text,url
0,MEXICO CITY Jan 2 Mexican businessman and bo...,2014-01-02,T,Mexican AT&T board member buys $20 mln stake i...,http://www.reuters.com//article/americamovil-s...
1,By Sinead Carew | NEW YORK NEW YORK AT&T I...,2014-01-03,T,AT&T pushes to lure T-Mobile customers; may he...,http://www.reuters.com//article/us-att-tmobile...
2,By Sinead CarewNEW YORK Jan 3 AT&T Inc on Fr...,2014-01-03,T,UPDATE 3-AT&T pushes to lure T-Mobile customer...,http://www.reuters.com//article/att-tmobile-id...
3,NEW YORK Jan 3 AT&T Inc on Friday offered cu...,2014-01-03,T,AT&T offers T-Mobile users up to $450 credit t...,http://www.reuters.com//article/att-tmobile-id...
4,By Sinead Carew | NEW YORK NEW YORK AT&T I...,2014-01-06,T,AT&T promises sponsored data services in comin...,http://www.reuters.com//article/us-ces-att-spo...
5,By Sinead Carew | NEW YORK NEW YORK Jan 6 ...,2014-01-06,T,AT&T promises sponsored data services in comin...,http://www.reuters.com//article/ces-att-sponso...
6,Carlyle Group LP (CG.O) said that former U.S...,2014-01-06,T,Carlyle Group hires former FCC chairman as dea...,http://www.reuters.com//article/us-carlyle-gen...
7,The escalating battle between No. 2 U.S. mob...,2014-01-07,T,AT&T kicks T-Mobile CEO out of its CES party,http://www.reuters.com//article/us-tmobile-att...
8,Jan 7 The escalating battle between No. 2 U....,2014-01-07,T,AT&T kicks T-Mobile CEO out of its CES party,http://www.reuters.com//article/tmobile-att-ej...
9,NEW YORK A deal that reduces the U.S. mobile...,2014-01-07,T,Verizon CEO says U.S. may demand spectrum sale...,http://www.reuters.com//article/us-verizon-mca...


In [89]:
links.to_csv('articles.csv', encoding="utf-8")

In [ ]:
news_links=[]
for sym in companies.Ticker_symbol:
    news_links.append(get_links_for_symbol(sym))

In [ ]:
day = date(2014, 1, 1)
# days=[]
# while day <= date(2016, 9, 1):
# # while day <= date(2014, 1, 11):
#     days.append(day.strftime('%m%d%Y')) if (day.weekday() not in [5, 6]) else 0
#     day+=timedelta(days=1)
# len(days)
(date.today() - date(2014, 1, 1)).days

In [ ]:
def get_links_for_symbol(symbol, start_date=date(2014, 1, 1)):
    
    links={}
    for day in [start_date + timedelta(n) for n in range((date.today() - start_date).days)]:
        url="http://www.reuters.com/finance/stocks/companyNews?symbol={}&date={}".format(symbol,day)
        print url
        soup = BeautifulSoup(urllib2.urlopen(url), "lxml")
        news=soup.find_all('div', {"id":"companyNews"})

        for feature in news[0].find_all('h2'):
        #     print feature.prettify()
            link=feature.find('a')
            if link.has_attr('href'):
                links[link.text]="http://www.reuters.com/"+link['href']
        for feature in news[1].find_all('div', {"class":"feature"}):
        #     print feature.prettify()
            link=feature.find('a')
            if link.has_attr('href'):
                title=link.text.strip('UPDATE 1-')
                links[title]="http://www.reuters.com/"+link['href']
        sleep(1)
        return links
    #     print url

In [ ]:
for d in [s_d + timedelta(n) for n in range((date.today() - s_d).days)]:
    print d

In [ ]:
symbol='F.N'
s_d=date.today()-timedelta(20)
a=get_links_for_symbol(symbol, s_d)

In [ ]:
a

In [ ]:
links={}
for day in days:
    url="http://www.reuters.com/finance/stocks/companyNews?symbol=F.N&date="+day
    
    soup = BeautifulSoup(urllib2.urlopen(url), "lxml")
    news_1=soup.find_all('div', {"id":"companyNews"})
    
    for feature in news_1[0].find_all('h2'):
    #     print feature.prettify()
        link=feature.find('a')
        if link.has_attr('href'):
            links[link.text]="http://www.reuters.com/"+link['href']
    for feature in news_1[1].find_all('div', {"class":"feature"}):
    #     print feature.prettify()
        link=feature.find('a')
        if link.has_attr('href'):
            title=link.text.strip('UPDATE 1-')
            links[title]="http://www.reuters.com/"+link['href']
    sleep(1)
#     print url

In [ ]:
links

In [ ]:
li=[]
for key, value in links.items():
#     print key
    li.append({'title':key, 'link':value})

In [ ]:
li=pd.DataFrame(li)

In [ ]:
li

In [ ]:
li.to_csv('news_links.csv')